In [3]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import requests
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets

In [4]:
#Local Statistics Data Tables
def show_profit():
    print("Showing profit...")

    url = "https://datavenge-35c4e96eb954.herokuapp.com/datavengers/"
    userVal = {"user_id" : 10}
    response = requests.post(url+"warehouse/1/profit", json = userVal)
    warehouses = response.json()

    warehouses = {
        "profit": [-15226, 0, 0, -6160, 0, 0, 0, 0, 0, 0, 0, 713],
        "profit_year": [2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 3001]
    }

    df = pd.DataFrame(warehouses)

    plt.figure(figsize=(10, 5))
    plt.bar(df['profit_year'].astype(str), df['profit'], color='skyblue')
    plt.xlabel('Year')
    plt.ylabel('Profit')
    plt.title('Profit by Year')
    plt.xticks(rotation=45)
    plt.tight_layout()  # Adjusts plot to ensure everything fits without overlapping
    plt.show()

def show_top_expensive_racks():
    print("Showing top 5 expensive racks...")
    url = "https://datavenge-35c4e96eb954.herokuapp.com/datavengers/"
    userVal = {"user_id" : 10}
    response = requests.post(url+"warehouse/1/rack/expensive", json = userVal)
    warehouses = response.json()

    df = pd.DataFrame(warehouses)
    print(df)

def show_bottom_parts_type():
    print("Showing bottom 3 parts type/material...")
    url = "https://datavenge-35c4e96eb954.herokuapp.com/datavengers/"
    userVal = {"user_id" : 10}
    response = requests.post(url+"warehouse/1/rack/material", json = userVal)
    warehouses = response.json()

    df = pd.DataFrame(warehouses)
    print(df)

def show_racks_under_capacity():
    print("Showing top 5 racks under 25% capacity...")
    url = "https://datavenge-35c4e96eb954.herokuapp.com/datavengers/"
    userVal = {"user_id" : 10}
    response = requests.post(url+"warehouse/1/rack/lowstock", json = userVal)
    warehouses = response.json()

    df = pd.DataFrame(warehouses)
    print(df)

def show_top_suppliers_warehouse():
    print("Showing top 3 suppliers that supplied to the warehouse...")
    url = "https://datavenge-35c4e96eb954.herokuapp.com/datavengers/"
    userVal = {"user_id" : 10}
    response = requests.post(url+"warehouse/1/transaction/suppliers", json = userVal)
    warehouses = response.json()

    df = pd.DataFrame(warehouses)
    print(df)

def show_top_users_exchanges():
    print("Showing top 3 users that receives the most exchanges...")
    url = "https://datavenge-35c4e96eb954.herokuapp.com/datavengers/"
    userVal = {"user_id" : 10}
    response = requests.post(url+"warehouse/1/users/receivesmost", json = userVal)
    warehouses = response.json()

    df = pd.DataFrame(warehouses)
    print(df)

def show_top_days_smallest_incoming():
    print("Showing top 3 days with the smallest incoming transactions' cost...")
    url = "https://datavenge-35c4e96eb954.herokuapp.com/datavengers/"
    userVal = {"user_id" : 10}
    response = requests.post(url+"warehouse/1//transaction/leastcost", json = userVal)
    warehouses = response.json()

    df = pd.DataFrame(warehouses)
    print(df)

In [5]:
# Global Statistics Data Tables
def show_top_warehouses_racks():
    print("Showing top 10 warehouses with the most racks...")
    gstat_url = "https://datavenge-35c4e96eb954.herokuapp.com/datavengers/"
    gstat_route = ["most/rack"]
    
    for gstat in gstat_route:
        response = requests.get(gstat_url+gstat)
        res = response.json()
        df = pd.DataFrame(res)
        print(gstat)
        display(df)


def show_top_warehouses_incoming_transactions():
    print("Showing top 5 warehouse that has the most incoming transactions...")
    gstat_url = "https://datavenge-35c4e96eb954.herokuapp.com/datavengers/"
    gstat_route = ["most/incoming"]
    
    for gstat in gstat_route:
        response = requests.get(gstat_url+gstat)
        res = response.json()
        df = pd.DataFrame(res)
        print(gstat)
        display(df)

def show_top_warehouses_delivers_exchanges(): #aquii
    print("Showing top 5 warehouse that delivers the most exchanges...")
    gstat_url = "https://datavenge-35c4e96eb954.herokuapp.com/datavengers/"
    gstat_route = ["most/deliver"]
    
    for gstat in gstat_route:
        response = requests.get(gstat_url+gstat)
        res = response.json()
        df = pd.DataFrame(res)
        print(gstat)
        display(df)
        
def show_top_users_transactions():
    print("Showing top 3 users that made the most transactions...")
    gstat_url = "https://datavenge-35c4e96eb954.herokuapp.com/datavengers/"
    gstat_route = ["most/transactions"]
    
    for gstat in gstat_route:
        response = requests.get(gstat_url+gstat)
        res = response.json()
        df = pd.DataFrame(res)
        print(gstat)
        display(df)

def show_top_warehouses_least_outgoing_transactions(): #aquiiii
    print("Showing top 3 warehouses with the least outgoing transactions...")
    gstat_url = "https://datavenge-35c4e96eb954.herokuapp.com/datavengers/"
    gstat_route = ["least/outgoing"]
    
    for gstat in gstat_route:
        response = requests.get(gstat_url+gstat)
        res = response.json()
        df = pd.DataFrame(res)
        print(gstat)
        display(df)
    
def show_top_warehouses_cities():
    print("Showing top 3 warehouses cities with the most transactions...")
    gstat_url = "https://datavenge-35c4e96eb954.herokuapp.com/datavengers/"
    gstat_route = ["most/city"]
    
    for gstat in gstat_route:
        response = requests.get(gstat_url+gstat)
        res = response.json()
        df = pd.DataFrame(res)
        print(gstat)
        display(df)

In [6]:
#Transactions Data Tables      
def show_latest_to_oldest():
    print("Showing latest to oldest transactions...")

def show_oldest_to_latest():
    print("Showing oldest to latest transactions...")

In [7]:
#Prices Data Tables
def show_part_prices():
    print("Showing part prices...")

def show_parts_supplied_by_suppliers():
    print("Showing parts supplied by suppliers...")

In [8]:
# Dropdown Menus
local_stats_dropdown = widgets.Dropdown(
    options=['Profit', 'Top 5 Expensive Racks', 'Bottom 3 Parts Type/Material', 'Top 5 Racks Under 25% Capacity', 'Top 3 Suppliers per Warehouse', 'Top 3 Users With Exchanges', 'Top 3 Days Smallest Incoming'],
    description='Local Stats:',
)

global_stats_dropdown = widgets.Dropdown(
    options=['Top 10 warehouses with the most racks', 'Top 5 warehouses with the most incoming transactions', 'Top 5 warehouses that delivers the most exchanges','Top 3 users that made the most transaction','Top 3 warehouses with the least outgoing transactions', 'Top 3 warehouses cities with the most transactions'],
    description='Global Stats:',
)

transactions_dropdown = widgets.Dropdown(
    options=['Latest to Oldest', 'Oldest to Latest'],
    description='Transactions:',
)

parts_dropdown = widgets.Dropdown(
    options=['Part Prices', 'Parts Supplied by Suppliers'],
    description='Parts:',
)

In [9]:
# Create an output widget to display results
output = widgets.Output()

def on_local_stats_change(change):
    with output:
        clear_output(wait=True)
        if change['type'] == 'change' and change['name'] == 'value':
            if change['new'] == 'Profit':
                show_profit()
            elif change['new'] == 'Top 5 Expensive Racks':
                show_top_expensive_racks()
            elif change['new'] == 'Bottom 3 Parts Type/Material':
                show_bottom_parts_type()
            elif change['new'] == 'Top 5 Racks Under 25% Capacity':
                show_racks_under_capacity()
            elif change['new'] == 'Top 3 Suppliers per Warehouse':
                show_top_suppliers_warehouse()
            elif change['new'] == 'Top 3 Users With Exchanges':
                show_top_users_exchanges()
            elif change['new'] == 'Top 3 Days Smallest Incoming':
                show_top_days_smallest_incoming()


def on_global_stats_change(change):
    with output:
        clear_output(wait=True)
        if change['type'] == 'change' and change['name'] == 'value':
            if change['new'] == 'Top 10 warehouses with the most racks': #/most/rack
                show_top_warehouses_racks()
            elif change['new'] == 'Top 5 warehouses with the most incoming transactions': #/most/incoming
                show_top_warehouses_incoming_transactions()
            elif change['new'] == 'Top 5 warehouses that delivers the most exchanges': #/most/deliver
                show_top_warehouses_delivers_exchanges()
            elif change['new'] == 'Top 3 users that made the most transactions': #/most/transactions
                show_top_users_transactions()
            elif change['new'] == 'Top 3 warehouses with the least outgoing transactions': #/least/outgoing
                show_top_warehouses_least_outgoing_transactions()
            elif change['new'] == 'Top 3 warehouses cities with the most transactions': #/most/city
                show_top_warehouses_cities()

def on_transactions_change(change):
    with output:
        clear_output(wait=True)
        if change['type'] == 'change' and change['name'] == 'value':
            if change['new'] == 'Latest to Oldest':
                show_latest_to_oldest()
            elif change['new'] == 'Oldest to Latest':
                show_oldest_to_latest()

def on_parts_dropdown_change(change):
    with output:
        clear_output(wait=True)
        if change['type'] == 'change' and change['name'] == 'value':
            if change['new'] == 'Part Prices':
                show_part_prices()
            elif change['new'] == 'Parts Supplied by Suppliers':
                show_parts_supplied_by_suppliers()

In [10]:
# Register the event handlers
local_stats_dropdown.observe(on_local_stats_change, names='value')
global_stats_dropdown.observe(on_global_stats_change, names='value')
transactions_dropdown.observe(on_transactions_change, names='value')
parts_dropdown.observe(on_parts_dropdown_change, names='value')

# Layout the widgets
title_widget = widgets.HTML(
    value="<h2 style='text-align: center;'>Datavengers - Phase 3</h2>"
)
dropdowns_vbox = widgets.HBox([local_stats_dropdown, global_stats_dropdown, transactions_dropdown, parts_dropdown],layout=widgets.Layout(justify_content='center', width='100%'))
main_vbox = widgets.VBox([title_widget, dropdowns_vbox, output])

# Display the main layout
display(main_vbox)